In [ ]:
import csv
import networkx as nx

In [ ]:
# Data collected by Laura Miguel by scraping the FOSDEM archive websites
# from the previous years of 2007 to 2022, then pseudonymized

with open('../data/fosdem-tracks-pseudo.csv') as f:
    data = list(csv.DictReader(f))

In [ ]:
data[0], data[1063], data[5617]

In [ ]:
# Build the graph

G = nx.Graph()

for row in data:
    if not row['speakers_name']:
        continue
        
    track = row['track']
    year = row['year']
    
    G.add_node(track, part='track')
    
    for speaker in row['speakers_name'].split('|'):
        G.add_node(speaker, part='speaker')
        
        # Keeping info about last pair (most recent)
        if not G.has_edge(speaker, track):
            G.add_edge(speaker, track, count=1, year=year)
        else:
            G[speaker][track]['count'] += 1
            G[speaker][track]['year'] = year

G.order(), G.size()

In [ ]:
# Check this prolific contributor in recent years

G.nodes["Clarie Nikolai"], G["Clarie Nikolai"]

In [ ]:
# Explore this bipartite graph

from ipysigma import Sigma

Sigma(G,
      node_size=G.degree,
      node_size_range=(1, 10),
      node_color='part',
      node_border_color_from='node',
      edge_size='count'
     )

In [ ]:
# Let's compare nodes based on the years they were participating to FOSDEM

from ipysigma import SigmaGrid

grid = SigmaGrid(G,
                 node_size=G.degree,
                 node_size_range=(1, 10),
                 node_border_color_from='node',
                 node_halo_color_gradient='OrRd',
                 default_node_color='rgba(0, 0, 0, 0.0)',
                 edge_size='count'
                )

YEAR_2012 = lambda n: sum(1 for _s, _t, a in G.edges(n, data=True) if a['year'] == '2012')
YEAR_2022 = lambda n: sum(1 for _s, _t, a in G.edges(n, data=True) if a['year'] == '2022' )

grid.add(node_halo_size=YEAR_2012, node_halo_color=YEAR_2012, node_zindex=YEAR_2012)
grid.add(node_halo_size=YEAR_2022, node_halo_color=YEAR_2022, node_zindex=YEAR_2022)

In [ ]:
# Let's explore further by reprocessing our graph with pelote's toolkit

!pip install pelote

In [ ]:
from pelote import monopartite_projection

mono = monopartite_projection(G, 'track', metric='overlap')

In [ ]:
Sigma(mono,
      node_size=G.degree,
      node_size_range=(1, 10),
      node_metrics=['louvain'],
      node_color='louvain',
      node_border_color_from='node',
      edge_size='weight'
     )

In [ ]:
from pelote import crop_to_largest_connected_component

crop_to_largest_connected_component(mono)

In [ ]:
Sigma(mono,
      node_size=G.degree,
      node_size_range=(1, 10),
      node_metrics=['louvain'],
      node_color='louvain',
      node_border_color_from='node',
      edge_size='weight'
     )